# Library

In [1]:
import numpy as np
import pandas as pd

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score

from keras.utils.np_utils import to_categorical
from keras.models import load_model, Model, Sequential
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.layers import Dense, Embedding, SimpleRNN, Dropout

import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Preprocessing
### Load Data

In [2]:
df = pd.read_pickle('preprocessed_df.pkl')
df = pd.DataFrame(df[['detokenize','plot']])
df.head()

,detokenize,plot
0,the desperate hour lakewood salah cerita suara...,1
1,edisi males review singkat tonton libur dp des...,1
2,plot utama orang deserter pursuit buru wamil j...,1
3,film hereditary horror thrill midsommar gatau ...,0
4,batman manusiawi tarung nya sadis scene pursui...,1


### Vectorize

In [3]:
df['sent_len'] = df['detokenize'].apply(lambda x: len(x.split(" ")))
max_sequence_length = np.round(df['sent_len'].mean() + 2 * df['sent_len'].std()).astype(int)
max_sequence_length

32

In [4]:
MAX_NB_WORDS = 1e5
le = LabelEncoder()

reviews = df["detokenize"]
label = le.fit_transform(df['plot'])
label = np.asarray(to_categorical(label))

tokenizer = Tokenizer(num_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(reviews)
review_seq = np.array(tokenizer.texts_to_sequences(reviews), dtype=object)
review_seq = pad_sequences(review_seq, padding='post', maxlen=max_sequence_length)

word_index = tokenizer.word_index
print(f'Dictionary Size\t: {len(word_index)}')
print(f'Example Label\t:\n{label[:5]}')

Dictionary Size	: 17718
Example Label	:
[[0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 1. 0.]
 [0. 0. 1.]]


# Variables

In [5]:
embedding_dim = 300
vocab_size = min(MAX_NB_WORDS, len(word_index) + 1)
test_size_split = 0.1

# Split Data

In [6]:
x_train, x_test, y_train, y_test = (
    train_test_split(review_seq, 
                     label, 
                     test_size=test_size_split, 
                     stratify = label, 
                     random_state=42)
)

x_train = tf.constant(x_train)
x_test = tf.constant(x_test)

y_train = tf.constant(y_train)
y_test = tf.constant(y_test)

print(f'Train\t| X shape: {x_train.shape}\tY shape: {y_train.shape}')
print(f'Test\t| X shape: {x_test.shape}\tY shape: {y_test.shape}')

Train	| X shape: (15522, 32)	Y shape: (15522, 3)
Test	| X shape: (1725, 32)	Y shape: (1725, 3)


# Model RNN

In [7]:
def RNN(x_train_c, y_train_c, x_test_c, y_test_c):
    rnn = Sequential()
    rnn.add(Embedding(vocab_size, 
                      embedding_dim, 
                      input_length=max_sequence_length,
                      trainable=False))
    rnn.add(SimpleRNN(units=256, activation='relu', recurrent_dropout=0.2, return_sequences=True))
    rnn.add(Dropout(0.2))
    rnn.add(SimpleRNN(units=128, activation='relu', return_sequences=True))
    rnn.add(Dropout(0.2))
    rnn.add(SimpleRNN(units=64, activation='relu'))
    rnn.add(Dropout(0.2))
    rnn.add(Dense(units=3, activation='softmax'))
    
    rnn.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    early_stopping = EarlyStopping(monitor='val_loss', mode = 'min', verbose = 1, patience=3, min_delta=0.0001)
    model_checkpoint = ModelCheckpoint('rnn_baseline_plot.h5', monitor = 'val_accuracy', mode = 'max', verbose = 1, save_best_only = True)
    
    history = rnn.fit(x_train_c, y_train_c, epochs= 10, batch_size=128, 
                      validation_data=(x_test_c, y_test_c), verbose = 1,
                      callbacks=[early_stopping, model_checkpoint])
    
    model = load_model('rnn_baseline_plot.h5')
    y_pred = model.predict(x_test_c)
    y_pred_cat = y_pred.argmax(axis=1)
    y_test_cat = np.argmax(y_test_c, axis=1)
   
    cm = confusion_matrix(y_test_cat, y_pred_cat)
    classreport = classification_report(y_test_cat, y_pred_cat)
    f1 = f1_score(y_test_cat, y_pred_cat,  average="macro")
    recall = recall_score(y_test_cat, y_pred_cat, average="macro")
    precision = precision_score(y_test_cat, y_pred_cat, average="macro")
    accuracy = accuracy_score(y_test_cat, y_pred_cat)

    print(classreport)
    print("F1 Score : ", f1)
    print("Precision : ", precision)
    print("Recall : ", recall)
    print("Accuracy : ", accuracy)

    return [f1, precision, recall, accuracy, cm]

# Running

In [8]:
hasil = []

for i in range(1, 6):
    hasil.append([str(i)] + RNN(x_train, y_train, x_test, y_test))

Epoch 1/10
122/122 [==============================] - ETA: 0s - loss: 1.0688 - accuracy: 0.3963
Epoch 1: val_accuracy improved from -inf to 0.40580, saving model to rnn_baseline_plot.h5
122/122 [==============================] - 27s 181ms/step - loss: 1.0688 - accuracy: 0.3963 - val_loss: 1.0646 - val_accuracy: 0.4058
Epoch 2/10
122/122 [==============================] - ETA: 0s - loss: 1.0655 - accuracy: 0.3984
Epoch 2: val_accuracy did not improve from 0.40580
122/122 [==============================] - 24s 200ms/step - loss: 1.0655 - accuracy: 0.3984 - val_loss: 1.0632 - val_accuracy: 0.4058
Epoch 3/10
122/122 [==============================] - ETA: 0s - loss: 1.0630 - accuracy: 0.4026
Epoch 3: val_accuracy did not improve from 0.40580
122/122 [==============================] - 26s 210ms/step - loss: 1.0630 - accuracy: 0.4026 - val_loss: 1.0634 - val_accuracy: 0.3971
Epoch 4/10
122/122 [==============================] - ETA: 0s - loss: 1.0615 - accuracy: 0.4031
Epoch 4: val_accuracy 

C:\Users\Alienware 15R2\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Alienware 15R2\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Alienware 15R2\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Alienware 15R2\anac

Epoch 1/10
122/122 [==============================] - ETA: 0s - loss: 1.0675 - accuracy: 0.3965
Epoch 1: val_accuracy improved from -inf to 0.40580, saving model to rnn_baseline_plot.h5
122/122 [==============================] - 37s 231ms/step - loss: 1.0675 - accuracy: 0.3965 - val_loss: 1.0644 - val_accuracy: 0.4058
Epoch 2/10
122/122 [==============================] - ETA: 0s - loss: 1.0651 - accuracy: 0.4040
Epoch 2: val_accuracy did not improve from 0.40580
122/122 [==============================] - 25s 203ms/step - loss: 1.0651 - accuracy: 0.4040 - val_loss: 1.0628 - val_accuracy: 0.4058
Epoch 3/10
122/122 [==============================] - ETA: 0s - loss: 1.0624 - accuracy: 0.4036
Epoch 3: val_accuracy did not improve from 0.40580
122/122 [==============================] - 22s 177ms/step - loss: 1.0624 - accuracy: 0.4036 - val_loss: 1.0598 - val_accuracy: 0.4058
Epoch 4/10
122/122 [==============================] - ETA: 0s - loss: 1.0636 - accuracy: 0.4011
Epoch 4: val_accuracy 

C:\Users\Alienware 15R2\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Alienware 15R2\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Alienware 15R2\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Alienware 15R2\anac

              precision    recall  f1-score   support

           0       0.00      0.00      0.00       368
           1       0.53      0.75      0.62       657
           2       0.61      0.70      0.65       700

    accuracy                           0.57      1725
   macro avg       0.38      0.48      0.42      1725
weighted avg       0.45      0.57      0.50      1725

F1 Score :  0.42471189773844636
Precision :  0.3811599099099099
Recall :  0.48241429296223814
Accuracy :  0.568695652173913
Epoch 1/10
122/122 [==============================] - ETA: 0s - loss: 1.0689 - accuracy: 0.3967
Epoch 1: val_accuracy improved from -inf to 0.40638, saving model to rnn_baseline_plot.h5
122/122 [==============================] - 36s 219ms/step - loss: 1.0689 - accuracy: 0.3967 - val_loss: 1.0694 - val_accuracy: 0.4064
Epoch 2/10
122/122 [==============================] - ETA: 0s - loss: 1.0657 - accuracy: 0.3967
Epoch 2: val_accuracy did not improve from 0.40638
122/122 [===================

C:\Users\Alienware 15R2\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Alienware 15R2\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Alienware 15R2\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Alienware 15R2\anac

Epoch 1/10
122/122 [==============================] - ETA: 0s - loss: 1.0689 - accuracy: 0.3985
Epoch 1: val_accuracy improved from -inf to 0.40580, saving model to rnn_baseline_plot.h5
122/122 [==============================] - 39s 240ms/step - loss: 1.0689 - accuracy: 0.3985 - val_loss: 1.0633 - val_accuracy: 0.4058
Epoch 2/10
122/122 [==============================] - ETA: 0s - loss: 1.0650 - accuracy: 0.4023
Epoch 2: val_accuracy did not improve from 0.40580
122/122 [==============================] - 21s 173ms/step - loss: 1.0650 - accuracy: 0.4023 - val_loss: 1.0626 - val_accuracy: 0.4058
Epoch 3/10
122/122 [==============================] - ETA: 0s - loss: 1.0632 - accuracy: 0.4058
Epoch 3: val_accuracy did not improve from 0.40580
122/122 [==============================] - 21s 174ms/step - loss: 1.0632 - accuracy: 0.4058 - val_loss: 1.0644 - val_accuracy: 0.4058
Epoch 4/10
122/122 [==============================] - ETA: 0s - loss: 1.0621 - accuracy: 0.4052
Epoch 4: val_accuracy 

C:\Users\Alienware 15R2\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Alienware 15R2\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Alienware 15R2\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Alienware 15R2\anac

Epoch 1/10
122/122 [==============================] - ETA: 0s - loss: 1.0700 - accuracy: 0.3970
Epoch 1: val_accuracy improved from -inf to 0.40580, saving model to rnn_baseline_plot.h5
122/122 [==============================] - 29s 167ms/step - loss: 1.0700 - accuracy: 0.3970 - val_loss: 1.0651 - val_accuracy: 0.4058
Epoch 2/10
122/122 [==============================] - ETA: 0s - loss: 1.0641 - accuracy: 0.4021
Epoch 2: val_accuracy did not improve from 0.40580
122/122 [==============================] - 18s 149ms/step - loss: 1.0641 - accuracy: 0.4021 - val_loss: 1.0694 - val_accuracy: 0.4058
Epoch 3/10
122/122 [==============================] - ETA: 0s - loss: 1.0633 - accuracy: 0.4037
Epoch 3: val_accuracy did not improve from 0.40580
122/122 [==============================] - 17s 143ms/step - loss: 1.0633 - accuracy: 0.4037 - val_loss: 1.0631 - val_accuracy: 0.4058
Epoch 4/10
122/122 [==============================] - ETA: 0s - loss: 1.0612 - accuracy: 0.4083
Epoch 4: val_accuracy 

C:\Users\Alienware 15R2\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Alienware 15R2\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Alienware 15R2\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Alienware 15R2\anac

# Save Output

In [9]:
df = pd.DataFrame(hasil, columns =['iterasi', 'f1', 'precision', 'recall', 'accuracy', 'cm'])
#df.to_excel('rnn_baseline_plot.xlsx')
print(df['f1'].mean())
print(df['accuracy'].mean())
df.head()

0.3465474750022177
0.49750724637681154


,iterasi,f1,precision,recall,accuracy,cm
0,1,0.401599,0.382002,0.458922,0.537971,"[[0, 295, 73], [0, 546, 111], [0, 318, 382]]"
1,2,0.424712,0.381160,0.482414,0.568696,"[[0, 226, 142], [0, 490, 167], [0, 209, 491]]"
2,3,0.294572,0.312150,0.364823,0.423188,"[[0, 322, 46], [0, 552, 105], [0, 522, 178]]"
3,4,0.399851,0.389831,0.462176,0.551304,"[[0, 109, 259], [0, 299, 358], [0, 48, 652]]"
4,5,0.212004,0.288259,0.334526,0.406377,"[[0, 5, 363], [0, 23, 634], [0, 22, 678]]"
